<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex04notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex04notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


----
## ロジスティック回帰＋勾配法によるパラメータの最適化 (1) 2クラス識別のロジスティック回帰
----




これまでに説明した最短距離法や最近傍法とはだいぶん仕組みの異なる識別の手法として，**ロジスティック回帰** (Logistic Regression) を取り上げます．
名前に「回帰」とついていますが，回帰／識別という分け方でいえば識別のための手法です．

----
### 準備


以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

----
### 2クラス識別のためのロジスティック回帰



簡単のため，まずは識別すべきクラスが2つしかない場合に限定してロジスティック回帰モデルを説明します．
以下，2つのクラスを $0$ と $1$ で表すことにして，$D$ 次元のデータ $\mathbf{x} = (x_1, x_2, \ldots, x_D)$ に対する正解クラスを $y\in \{0, 1\}$ で表すことにします．



#### シグモイド関数

というわけでロジスティック回帰モデルの説明に入りたいところですが，まずは準備...．次のような関数を考えます．

$$
\sigma(s) = \frac{1}{1+\exp{(-s)}}\qquad (1)
$$

これは，**シグモイド関数**（sigmoid function）と呼ばれるものです．



In [ ]:
# シグモイド関数の値を計算
xmin, xmax = -6, 6
X = np.linspace(xmin, xmax, num=100)
Y = 1/(1+np.exp(-X))

# グラフに描く
fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)
ax.set_xlim(xmin, xmax)
ax.set_ylim(-0.1, 1.1)
ax.axhline(y=0, color='black', linestyle='-')
ax.axvline(x=0, color='black', linestyle='-')
ax.axhline(y=1, color='gray', linestyle='--')
ax.plot(X, Y, linewidth=2, label='$\sigma(s)$')
ax.legend()
plt.show()

式からもグラフからもわかるように，シグモイド関数には
$$
\lim_{s \rightarrow -\infty}\sigma(s) = 0,\qquad
\lim_{s \rightarrow +\infty}\sigma(s) = 1,\qquad
\sigma(0) = \frac{1}{2}
$$
という性質があります．

#### ロジスティック回帰モデル

$D$次元のデータ $\mathbf{x}$ を入力として受け取るロジスティック回帰モデルは，シグモイド関数を使って次のような式で表されます．

$$
\begin{aligned}
f(\mathbf{x}) &= \sigma(w_0 + w_1x_1+\cdots + w_Dx_D) = \sigma\left(w_0 + \sum_{d=1}^{D}w_dx_d \right) \\
&= \frac{1}{1+\exp{\left( - \left( w_0 + \sum_{d=1}^{D}w_dx_d \right) \right)}} \qquad (2)
\end{aligned}
$$

パラメータは，$w_0, w_1, \ldots, w_D$ の $(D+1)$ 個あります．
ちなみに，シグモイド関数の括弧の中は，平面当てはめのモデルと全く同じ式になっています．



シグモイド関数でできていることから分かるように，このモデルの出力は $ 0 < f(\mathbf{x}) < 1$ を満たします．
そこで，出力の値が $0$ に近いか $1$ に近いかによって 2 クラスを識別することができます（典型的には，$\frac{1}{2}$ との大小を比較します）．

### 2次元データの例

上記の式だけではイメージするのが難しいです．また，$D$次元の話のままではグラフに描いたりするのも困難です．そこで，$D=2$ の場合について，人工的に作ったデータを例にして説明します．

In [ ]:
## 2次元正規分布で2クラスのデータを生成する関数

def getData(seed = None):

    if seed != None:
        np.random.seed( seed )

    # two 2-D spherical Gaussians
    X0 = 1.0*np.random.randn(200, 2) + [3.0, 3.0]
    X1 = 1.0*np.random.randn(200, 2) + [7.0, 6.0]
    X  = np.vstack((X0, X1))
    lab0 = np.zeros(X0.shape[0], dtype=int)
    lab1 = np.zeros(X1.shape[0], dtype=int) + 1
    label = np.hstack((lab0, lab1))
 
    return X, label

入力データが $(x_1, x_2)$ であるとき，このデータの所属クラスの正解を $y$ とおくと，$y$ は $0$ または $1$ です．
以下の左の散布図は，いままで通り $(x_1, x_2)$ 平面に学習データを色分けして描いたものです．青い点は所属クラスが$0$の学習データ，オレンジの点は所属クラスが$1$の学習データです．
一方，右の3次元散布図は，$(x_1, x_2, y)$ の値を描いたものです．
青い点たちが $y = 0$ の平面上にいるのに対して，オレンジの点たちは $y=1$ の平面上にいます．そのため，このグラフではオレンジの点たちが浮いています．

In [ ]:
X, lab = getData()

fig = plt.figure(facecolor='white', figsize=(14, 6))

# 左の2次元散布図
ax0 = fig.add_subplot(121)
ax0.set_xlim(0, 10)
ax0.set_ylim(0, 10)
ax0.set_aspect('equal')
ax0.scatter(X[lab == 0, 0], X[lab == 0, 1]) # blue
ax0.scatter(X[lab == 1, 0], X[lab == 1, 1]) # orange
ax0.set_xlabel('$x_1$')
ax0.set_ylabel('$x_2$')

# 右の3次元散布図
elevation = 20
azimuth = -70
ax1 = fig.add_subplot(122, projection='3d')
ax1.scatter(X[lab==0, 0], X[lab==0, 1], 0) # blue
ax1.scatter(X[lab==1, 0], X[lab==1, 1], 1) # orange
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.view_init(elevation, azimuth)
ax1.set_xlabel('$x_1$')
ax1.set_ylabel('$x_2$')
ax1.set_zlabel('$y$')

plt.show()

次に，これらの学習データを用いてロジスティック回帰の学習を行った結果を示します（学習をどのように行うか，パラメータをどのように決めるか，については後で説明します）．
緑の網目で表された曲面は，様々な $(x_1, x_2)$ の値に対して計算したモデルの出力を表しています．
学習データの青い点，オレンジの点のそれぞれにうまく当てはまる形をしていることがわかります．
また，以下のセルをそのまま実行した場合，$(x_1, x_2) = (4, 4)$ という入力に対する出力の値を計算して表示させるとともに，赤い点として描画しています．
出力の値は $0.285$ であり，このデータは $y = 0$ の方のクラス（青い方）に識別されると考えられます．

In [ ]:
#@title (x1, x2) の値をいろいろ変えてセルを実行してみよう

w0, w1, w2 = -8.2, 1.1, 0.72

x1 = 4.0 #@param {type: 'number'}
x2 = 4.0 #@param {type: 'number'}

# 入力 (x1, x2) に対する出力を計算
z = 1.0/(1+np.exp(-(w0 + w1*x1 + w2*x2)))

# 等間隔にデータを作ってモデル出力を計算
xx1, xx2 = np.meshgrid(np.linspace(0, 10, num=16), np.linspace(0, 10, num=16))
XX = np.vstack((xx1.ravel(), xx2.ravel())).T
ZZ = 1.0/(1+np.exp(-(w0 + w1*XX[:, 0] + w2*XX[:, 1]))) # モデル出力を計算
zz = ZZ.reshape(xx1.shape)

# 3次元プロットの視点の設定
elevation = 20 # 上下方向の角度
azimuth = -70  # 左右方向の角度

# 3次元プロット
fig = plt.figure(facecolor='white', figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[lab==0, 0], X[lab==0, 1], 0)
ax.scatter(X[lab==1, 0], X[lab==1, 1], 1)
ax.plot_wireframe(xx1, xx2, zz, color='green')
ax.scatter(x1, x2, z, s=100, color='red')
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.view_init(elevation, azimuth)
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.set_zlabel('$y$')
plt.show()

# (x1, x2) に対する出力
print(f'入力 ({x1}, {x2}) に対する出力: {z:.3f}')

#### ★★ やってみよう ★★

1. 上のセルの `x1, x2` の値を適当に変えて，どのような識別をしているか観察しなさい．
1. 下のセルに以下のパラメータの組のそれぞれを入力して実行し，どの組が最もこのデータをうまく識別できるか考えましょう．
これらのうち最もうまく識別できそうなものの記号をノート等（紙媒体）にメモしておこう．
    - (a) `(w0, w1, w2) = ( 12,  1.5, -1)`
    - (b) `(w0, w1, w2) = (-12,  1.5,  1)`
    - (c) `(w0, w1, w2) = ( 12, -1.5, -1)`
    - (d) `(w0, w1, w2) = (-12, -1.5,  1)`


In [ ]:
#@title (w0, w1, w2) の値をいろいろ変えてセルを実行してみよう

# パラメータの例
#w0, w1, w2 = -8.2, 1.1, 0.72

w0 =  -8.2#@param {type: 'number'}
w1 =  1.1#@param {type: 'number'}
w2 =  0.72#@param {type: 'number'}

# 等間隔にデータを作ってモデル出力を計算
xx1, xx2 = np.meshgrid(np.linspace(0, 10, num=16), np.linspace(0, 10, num=16))
XX = np.vstack((xx1.ravel(), xx2.ravel())).T
ZZ = 1.0/(1+np.exp(-(w0 + w1*XX[:, 0] + w2*XX[:, 1]))) # モデル出力を計算
zz = ZZ.reshape(xx1.shape)

# 3次元プロットの視点の設定
elevation = 20 # 上下方向の角度
azimuth = -70  # 左右方向の角度

# 3次元プロット
fig = plt.figure(facecolor='white', figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[lab==0, 0], X[lab==0, 1], 0)
ax.scatter(X[lab==1, 0], X[lab==1, 1], 1)
ax.plot_wireframe(xx1, xx2, zz, color='green')
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.view_init(elevation, azimuth)
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.set_zlabel('$y$')
plt.show()

----
### 交差エントロピー


ロジスティック回帰モデルのパラメータはどうやって決めたらよいのでしょうか？
最小二乗法の場合，パラメータを決めるための規準として「学習データに対する二乗誤差」を考え，それを最小化するようにパラメータを決めていました．式(2)のロジスティック回帰の場合は，以下に述べる **交差エントロピー** (cross entropy, 注) の最小化を考えます．

<span style="font-size: 75%">
※注: この「交差エントロピー」は，「情報理論」という分野で登場する概念です．
二つの確率分布の間の近さを表す尺度です．
ロジスティック回帰モデルは確率モデルの一種であり，その出力が確率を表すものとして，その確率分布が正解の確率分布に近づくように学習している，というような話が裏に隠れています．が，この授業ではその辺りのことは説明しません．
</span>

> $N$ 個のデータ
$$
(\mathbf{x}_1, y_1), (\mathbf{x}_2, y_2), (\mathbf{x}_N, y_N)
$$
> が与えられるとする．$\mathbf{x}_n \in {\cal R}^{D}$ は入力データであり，$y_n \in \{0, 1\}$ はこのデータの所属クラスの正解を表す値である（$n=1,2,\ldots,N$）．
> 
> このとき，式(2)のロジスティック回帰モデルの交差エントロピー $H$ は次式のようになる（注）．
> 
> $$
\begin{aligned}
H &= -\sum_{n=1}^{N} \left( y_n\log{f(\mathbf{x}_n})+(1-y_n)\log{\left( 1-f(\mathbf{x}_n)\right)} \right) \qquad (3)
\end{aligned}
$$


式(3)右辺の $\sum$ の中のものを $\ell_n$ とおくと，その値は，

$$
\ell_n = \left\{ \begin{array}{ll}
\log{f(\mathbf{x}_n}) & \mbox{$y_n = 1$ のとき}\\
\log{\left( 1-f(\mathbf{x}_n)\right)} & \mbox{$y_n = 0$ のとき}\\
\end{array} \right.
$$

となります．$0 < f(\mathbf{x}) < 1$ であることに注意して考えると，$\log{f(\mathbf{x}})$ の値は，$f(\mathbf{x})$ が大きくなって $1$ に近づけば近づくほど大きくなることがわかります．一方，$\log{\left( 1-f(\mathbf{x})\right)}$ の値は，$f(\mathbf{x})$ が小さくなって $0$ に近づけば近づくほど大きくなります．つまり，$\ell_n$ の値は，$f(\mathbf{x}_n)$ が $y_n$ に近くなる（正解に近づく）ほど大きくなります．

したがって，交差エントロピー $H$ の値は，個々の学習データに対するモデルの出力 $f(\mathbf{x}_n)$ が正解 $y_n$ に近づくほど小さくなります（右辺の先頭に負号が付いてることに注意）．
ロジスティック回帰では，この交差エントロピーが小さくなるようにパラメータを定めます（どうやって？の話はまた後で）．

----
### **ロジスティック回帰の問題設定（2クラスの場合）**

ここまでの説明をふまえて，2クラス識別のためのロジスティック回帰の問題設定を示しておきます．



**［ロジスティック回帰の問題設定（2クラスの場合）］**

$D$次元のデータを二つのクラスに識別するモデルを学習させる．学習データは $N$ 個あり，次のように与えられる．

$$
(\mathbf{x}_1, y_1), (\mathbf{x}_2, y_2),\ldots , (\mathbf{x}_N, y_N)
$$

ただし，$\mathbf{x}_n \in {\cal R}^{D}$ はモデルへの入力であり，$y_n \in \{0, 1\}$ はこのデータの所属クラスの正解を表す値である（$n=1,2,\ldots,N$）．

学習モデルは次式で定める．
$$
f(\mathbf{x}) = \frac{1}{1+\exp{\left( - \left( w_0 + \sum_{d=1}^{D}w_dx_d \right) \right)}}
$$
このモデルのパラメータは $w_0, w_1, \ldots, w_D$ の $(D+1)$ 個ある．

このとき，モデルの出力と正解の値との間の「遠さ」を，次式の交差エントロピーで定義する．
$$
\begin{aligned}
H &= -\sum_{n=1}^{N} \left( y_n\log{f(\mathbf{x}_n})+(1-y_n)\log{\left( 1-f(\mathbf{x}_n)\right)} \right) \qquad (3)
\end{aligned}
$$
この $H$ の値がなるべく小さくなるようにパラメータ $w_0, w_1, \ldots, w_D$ を求めたい．

最小二乗法のときは，二乗誤差をパラメータで微分して $=0$ とおくことで，最適なパラメータが満たす方程式を導出できました．
しかし，上記の場合は，交差エントロピーがより複雑な式をしているため，そのような手は使えません．
ではどうするのか，という話はまた後で．